<a href="https://colab.research.google.com/github/faizankshaikh/AV_Article_Codes/blob/master/Inception_v1_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

import cv2 
import math
import numpy as np 
import tensorflow as tf
import keras.backend as K

from keras.models import Model
from keras.optimizers import SGD 
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.layers.core import Layer
from keras.callbacks import LearningRateScheduler
from keras.initializers import glorot_uniform, Constant
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate,\
    GlobalAveragePooling2D, AveragePooling2D, Flatten

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_valid, y_valid) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, 10)
y_valid = np_utils.to_categorical(y_valid, 10)

def CustomImageDataGenerator(X, y, batch_size):
  generator = ImageDataGenerator(
    rescale=1/255.,
    dtype='float32')
  
  datagen = generator.flow(X, y, batch_size=batch_size)

  while True:
    X, y = datagen.__next__()
    X = np.array([cv2.resize(img, (224,224)) for img in X[:,:,:,:]])
    yield X, [y, y, y]

In [0]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(
        filters=filters_1x1, 
        kernel_size=(1, 1), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(x)
    
    conv_3x3 = Conv2D(
        filters=filters_3x3_reduce, 
        kernel_size=(1, 1), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(x)

    conv_3x3 = Conv2D(
        filters=filters_3x3, 
        kernel_size=(3, 3), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(conv_3x3)

    conv_5x5 = Conv2D(
        filters=filters_5x5_reduce, 
        kernel_size=(1, 1), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(x)

    conv_5x5 = Conv2D(
        filters=filters_5x5, 
        kernel_size=(5, 5), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(conv_5x5)

    pool_proj = MaxPool2D(
        pool_size=(3, 3), 
        strides=(1, 1), 
        padding='same'
        )(x)

    pool_proj = Conv2D(
        filters=filters_pool_proj, 
        kernel_size=(1, 1), 
        padding='same', 
        activation='relu', 
        kernel_initializer=kernel_init, 
        bias_initializer=bias_init
        )(pool_proj)

    output = concatenate(
        inputs=[conv_1x1, conv_3x3, conv_5x5, pool_proj], 
        axis=3, 
        name=name
        )
    
    return output

In [0]:
kernel_init = glorot_uniform()
bias_init = Constant(value=0.2)

In [5]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [0]:
model.summary()

In [7]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [0]:
model.fit_generator(CustomImageDataGenerator(X_train, y_train,
                        batch_size=256),
                        epochs=epochs,
                        steps_per_epoch=200,
                        use_multiprocessing=True,
                        workers=4,
                        callbacks=[lr_sc]    
                    )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/25



/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'







Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
200/200 [==============================] - 218s 1s/step - loss: 3.7005 - output_loss: 2.3221 - auxilliary_output_1_loss: 2.2912 - auxilliary_output_2_loss: 2.3033 - output_acc: 0.1143 - auxilliary_output_1_acc: 0.1238 - auxilliary_output_2_acc: 0.1117
Epoch 2/25

Epoch 00002: LearningRateScheduler setting learning rate to 0.01.
200/200 [==============================] - 195s 974ms/step - loss: 3.2905 - output_loss: 2.0650 - auxilliary_output_1_loss: 2.0379 - auxilliary_output_2_loss: 2.0472 - output_acc: 0.2155 - auxilliary_output_1_acc: 0.2431 - auxilliary_output_2_acc: 0.2337
Epoch 3/25

Epoch 00003: LearningRateScheduler setting learning rate to 0.01.
 48/200 [======>.......................] - ETA: 2:28 - loss: 3.1024 - output_loss: 1.9481 - auxilliary_output_1_loss: 1.9233 - auxilliary_output_2_loss: 1.9243 - output_acc: 0.2620 - auxilliary_output_1_acc: 0.2888 - auxilliary_output_2_acc: 0.2900